# Hands on : Introduction to BDT on HEP dataset

1. Load data from root (make sure packages load!)
2. Explore the data and weights
3. Preprocess data for training
4. Train a Boosted Decision Tree
5. Quantify its performance

This tutorial will probably be a bit more hands-on, focusing a bit more on the data and preprocessing. The Neural Network tutorial later today will give more time to play around with training. Feel free to revisit this notebook later today! 

### Many thanks to _Fernando Acosta_, _David Rousseau, Yann Coadou_ for their contributions.

## Import Packages

In [ ]:
import os
import numpy as np
import pandas as pd
import uproot as ur

import matplotlib.pyplot as plt
from IPython.display import display, HTML

%matplotlib inline
import time

pd.set_option('display.max_columns', 100)  # to see more columns of df.head()
np.random.seed(31415)  # set the np random seed for the reproducibility

# some utilities
from math import sqrt
from math import log

In [ ]:
# import xgboost

#print(xgboost.__version__)  # Tested with 1.6.1, version above 1 is recommended.
import lightgbm

print(lightgbm.__version__)  # Tested with 2.2.3
import sklearn

print(sklearn.__version__)  # Tested with 1.0.2

# Load events

[created from [ATLAS Open Data](http://opendata.atlas.cern/release/2020/documentation/datasets/intro.html)]

In [ ]:
filename = "dataWW_d1.root"
file = ur.open(filename)
print(file.classnames())

In [ ]:
tree = file["tree_event"]
dfall = tree.arrays(library="pd")
print("File loaded with ", dfall.shape[0], " events ")

In [ ]:
# shuffle the events, already done but just to be safe!
dfall = dfall.sample(frac=1).reset_index(drop=True)
from datetime import datetime

print("now :", datetime.now())
print("File loaded with ", dfall.shape[0], " events ")

At this point, it should tell you "File Loaded with XXX events". If not, it could not access the datafile. No point going further!

# Examine dataset

In [ ]:
# dump list of features
dfall.columns

In [ ]:
dfall.mcWeight *= 4  # arbitrary scale to have larger significance

In [ ]:
# examine first few events
display(dfall.head())
display(dfall.head())

In [ ]:
# examine feature distribution
dfall.describe()

In [ ]:
label_weights = (dfall[dfall.label == 0].mcWeight.sum(), dfall[dfall.label == 1].mcWeight.sum())
print("sum of label weights  Background, Signal =", label_weights)

label_nevents = (dfall[dfall.label == 0].shape[0], dfall[dfall.label == 1].shape[0])
print("total class number of events B S", label_nevents)

## Event selection

This notebook essentially tries to classify events containing a Higgs Boson.

The simulation includes top-quark-pair production, single-top production, production of weak bosons in association with jets ($W$+jets, $Z$+jets), production of a pair of bosons (diboson $WW, WZ, ZZ$) and __SM Higgs ($\to WW$)__ production.

We will only keep events with exactly two leptons __dfall.lep_n==2__

In [ ]:
print("Df shape before selection :", dfall.shape)

# Sometimes we keep only events with positive weight. This is in principle wrong.
# Many Data Science tools break given a negative weight, like XGBoost, but LightGBM can handle it.
# fulldata = dfall[(dfall.lep_n == 2) & (dfall.mcWeight > 0)]
fulldata = dfall[(dfall.lep_n == 2)]


print("Df shape after selection :", fulldata.shape)

___


### Try not to change the cells above $\uparrow$
...and return to this cell (or rerun the whole notebook) after changing things below.

___

# Explore the Data

### Choose features to train on

In [ ]:
# WARNING : there should be no selection nor shuffling later on ! (otherwise misalignement)
target = fulldata["label"]
weights = fulldata["mcWeight"]


# for simplicity of the exercise only keep some features
data = pd.DataFrame(fulldata, columns=["met_et", "met_phi", "lep_pt_0", "lep_pt_1", 'lep_phi_0', 'lep_phi_1'])

print("Df shape of dataset to be used :", data.shape)
display(data.head())
display(target.head())
display(weights.head())

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Simple Histo of ET for pT sum > 1000
data[data.lep_pt_0 + data.lep_pt_1 > 1000]['met_et'].plot.hist(
    bins=np.linspace(0, 400, 100), ax=axes[0], title='Missing Transverse Energy for large lepton Pt'
)


# Scatter of pT vs. ET
fig = plt.figure()
data[target == 0].plot.scatter(x='met_et', y='lep_pt_0', color="b", label="B", ax=axes[1])
data[target == 1].plot.scatter(x='met_et', y='lep_pt_0', color="r", label="S", ax=axes[1])
axes[1].set_title("Lepton $p_\mathrm{T}\ vs.\ Missing E_\mathrm{T}$")
plt.show()

In [ ]:
# Simple example of pandas array "slicing"
data[data.lep_pt_0 + data.lep_pt_1 > 2000].head()

## Examine the weights

In [ ]:
fig, ax = plt.subplots()
# fig=plt.figure()

bins = np.linspace(-1, 3, 101)
plt.hist(weights[target == 0] * 1000, bins=bins, color='b', alpha=0.5, density=True, label='[B]ackground')
plt.hist(weights[target == 1] * 1000, bins=bins, color='r', alpha=0.5, density=True, label='[S]ignal')
plt.legend(loc='best')
ax.set_xlabel('weight*1000')
plt.show()

# Some weight studies 

$s=\sum w$ for signal dataset : predicted number of signal events (luminosity, cross section, efficiencies etc... already includded in the weights). Ditto for background, $b$. 


Effective number of events fraction : $\frac{N_{eff}}{N}= \frac{1}{1+\frac{Var(w)}{<w>^2}}$ . Example : if 0.2 it means the precision achieved with this dataset is the one which would be achieved with an unweighted dataset of 0.2 x N events (this is a rough estimate, only true for a simple counting)

In [ ]:
label_n_weights = np.zeros(2)
label_sum_weights = np.zeros(2)
label_mean_weights = np.zeros(2)
label_std_weights = np.zeros(2)
label_neff_fraction = np.zeros(2)
label_sum_weightsSqr = np.zeros(2)

for i in range(2):
    label_n_weights[i] = weights[target == i].size
    label_mean_weights[i] = weights[target == i].mean()
    label_std_weights[i] = weights[target == i].std()
    label_sum_weights[i] = weights[target == i].sum()
    label_sum_weightsSqr[i] = (weights[target == i] **2).sum()
Neffective = label_sum_weights**2 / label_sum_weightsSqr

print("Weights quantities for background (target==0) and signal (target==1)")
print("Weights sum", np.round(label_sum_weights, 1))
print("N events", label_n_weights)
#print("Weights mean", label_mean_weights)
#print("Weights std", label_std_weights)
print("Effective number of events", np.round(Neffective, 1))

## Feature engineering (Two variations)
To be switched on in a second iteration

1. See if using more features improves model performance

In [ ]:
more_features = False
if more_features:
    data = pd.DataFrame(
        fulldata,
        columns=[
            "met_et",
            "met_phi",
            "lep_pt_0",
            "lep_pt_1",
            'lep_eta_0',
            'lep_eta_1',
            'lep_phi_0',
            'lep_phi_1',
            'jet_n',
            'jet_pt_0',
            'jet_pt_1',
            'jet_eta_0',
            'jet_eta_1',
            'jet_phi_0',
            'jet_phi_1',
        ],
    )

2. Engineer our own feature, $\Delta\varphi_l$

In [ ]:
do_feature_engineering = False
if do_feature_engineering:
    data["lep_deltaphi"] = np.abs(np.mod(data.lep_phi_1 - data.lep_phi_0 + 3 * np.pi, 2 * np.pi) - np.pi)
    # data["lep_deltaphi"]=data.lep_phi_1-data.lep_phi_0

    print(data.shape)
    display(data.head())

# Plot the features

In [ ]:
plt.figure()

ax = data[target == 0].hist(weights=weights[target == 0], figsize=(15, 12), color='b', alpha=0.5, density=True, label="B")
ax = ax.flatten()[: data.shape[1]]  # to avoid error if holes in the grid of plots (like if 7 or 8 features)
ax = data[target == 1].hist(
    weights=weights[target == 1], figsize=(15, 12), color='r', alpha=0.5, density=True, ax=ax, label="S"
)
# ax.legend(loc="best")

plt.legend(loc="best")
plt.show()

### Features correlation matrix

In [ ]:
import seaborn as sn  # seaborn for plots with more appealing defaults

fig, axes = plt.subplots(1, 2, sharex=True, figsize=(15, 5))
corrMatrix = data[target == 1].corr()
sn.heatmap(corrMatrix, annot=True, ax=axes[0])
axes[0].set_title("Signal Feature Correlation Matrix", fontsize=18)

# print ("Background feature correlation matrix")
corrMatrix = data[target == 0].corr()
sn.heatmap(corrMatrix, annot=True, ax=axes[1])
_ = axes[1].set_title("Background Feature Correlation Matrix", fontsize=18)

# Preprocess Data

## Split Data into Test and Training

In [ ]:
np.random.seed(31415)

from sklearn.model_selection import train_test_split

train_size = 0.75  # fraction of sample used for training

X_train, X_test, y_train, y_test, weights_train, weights_test = train_test_split(
    data, target, weights, train_size=train_size
)

# Reset index for dataseries, not needed for ndarray (X_train, X_test)
# Basically just re-adding the original element indexing from pandas
y_train, y_test, weights_train, weights_test = (
    y_train.reset_index(drop=True),
    y_test.reset_index(drop=True),
    weights_train.reset_index(drop=True),
    weights_test.reset_index(drop=True),
)

print("X_train shape:", X_train.shape)
print("y_train shape:", y_train.shape)
print("weights shape:", weights_train.shape, "\n")

print("X_test shape:", X_test.shape)
print("y_test shape:", y_test.shape)
print("weight shape:", weights_test.shape, "\n")

## Standardize the Data

**Scale to Mean of 0 and Variance of 1.0:**   $\ \ \ \ (x-\mu)/\sigma$

In [ ]:
# not usually needed for BDT but is good practice
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)  # applies the transformation calculated the line above

## Adjust the Test and Train Signal/Background Weights
Train on equal amount of Signal and Background, Test on 'natural' ratio

In [ ]:
class_weights_train = (weights_train[y_train == 0].sum(), weights_train[y_train == 1].sum())

for i in range(len(class_weights_train)):  # loop on B then S targets(labels)
    # training dataset: equalize number of background and signal
    weights_train[y_train == i] *= max(class_weights_train) / class_weights_train[i]

    # test dataset : increase test weight to compensate for sampling
    weights_test[y_test == i] *= 1 / (1 - train_size)

print("Weights have been normalised to a given number of proton collision")
print("Orig : total weight sig", weights[target == 1].sum())
print("Orig : total weight bkg", weights[target == 0].sum(), "\n")

print("Test : total weight sig", weights_test[y_test == 1].sum())
print("Test : total weight bkg", weights_test[y_test == 0].sum(), "\n")
print("Train : total weight sig", weights_train[y_train == 1].sum())
print("Train : total weight bkg", weights_train[y_train == 0].sum())

# Train with LightGBM (handles negative weighted events unlike XGBoost)

In [ ]:
np.random.seed(31415)  # set the random seed

import lightgbm as lgb
from sklearn.metrics import roc_auc_score  # for binary classification if x > 0.5 -> 1 else -> 0

# gbm = lgb.LGBMClassifier()
gbm = lgb.LGBMClassifier() # HPO, check on the web https://lightgbm.readthedocs.io/ for other parameters


starting_time = time.time()

gbm.fit(X_train, y_train.values, sample_weight=weights_train.values)


training_time = time.time() - starting_time
print("Training time:", training_time)

y_pred_gbm = gbm.predict_proba(X_test)[:, 1]
y_pred_gbm = y_pred_gbm.ravel()
y_pred_train_gbm = gbm.predict_proba(X_train)[:, 1].ravel()

In [ ]:
# easier to remove negative weighted events for fast AUC calucaltion
posWeightsTrain, posWeightsTest = weights_train>0, weights_test>0

auc_test_gbm = roc_auc_score(y_true=y_test[posWeightsTest], 
                             y_score=y_pred_gbm[posWeightsTest], 
                             sample_weight=weights_test[posWeightsTest])
print("auc test:", auc_test_gbm)
print("auc train:", roc_auc_score(y_true=y_train.values[posWeightsTrain], 
                                  y_score=y_pred_train_gbm[posWeightsTrain], 
                                  sample_weight=weights_train[posWeightsTrain]))
print("auc test without weights", roc_auc_score(y_true=y_test, y_score=y_pred_gbm))

## Significance Function

$\mathrm{med}[Z_0|1] = \sqrt{q_{0,A}} = \sqrt{2+((s+b)\ln(1+s/b)-s)}$

**asimov significance [arXiv:1007.1727](https://arxiv.org/pdf/1007.1727.pdf) [Eq. 97]**

asimov for significance. Need to esimate your sensitivity to MC. Need thousands of toy tests everytime you use simulation, you need to test your sensitivity. Running a toy MC thousands of times, should converge to 'truth'. Asimov is representative of. Number of sigmas.

In [ ]:
from math import sqrt
from math import log


def amsasimov(s, b):
    if b <= 0 or s <= 0:
        return 0
    try:
        return sqrt(2 * ((s + b) * log(1 + float(s) / b) - s))
    except ValueError:
        print(1 + float(s) / b)
        print(2 * ((s + b) * log(1 + float(s) / b) - s))
    # return s/sqrt(s+b)

In [ ]:
# sum S & B after certain thresholds
int_pred_test_sig_gbm = [weights_test[(y_test == 1) & (y_pred_gbm > th_cut)].sum() for th_cut in np.linspace(0, 1, num=50)]
int_pred_test_bkg_gbm = [weights_test[(y_test == 0) & (y_pred_gbm > th_cut)].sum() for th_cut in np.linspace(0, 1, num=50)]

vamsasimov_gbm = [amsasimov(sumsig, sumbkg) for (sumsig, sumbkg) in zip(int_pred_test_sig_gbm, int_pred_test_bkg_gbm)]
significance_gbm = max(vamsasimov_gbm)
Z = significance_gbm
print("Z:", Z)

In [ ]:
plt.plot(np.linspace(0, 1, num=50), vamsasimov_gbm, label='LGBM (Z = {})'.format(np.round(significance_gbm, decimals=2)))


plt.title("BDT Significance")
plt.xlabel("Threshold")
plt.ylabel("Significance")
plt.legend()
plt.savefig("Significance_gbm.pdf")
plt.show()

In [ ]:
from extra_functions import compare_train_test

In [ ]:
compare_train_test(
    y_pred_train_gbm,
    y_train,
    y_pred_gbm,
    y_test,
    xlabel="LGBM score",
    title="LGBM",
    weights_train=weights_train.values,
    weights_test=weights_test.values,
)

plt.savefig("Score_BDT_GBM_Hist.pdf")

# Hyper Parameter Optimisation
**Come back to this if done early**

- Can be done by hand or with [random search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.RandomizedSearchCV.html) or [grid search](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) .

In [ ]:
# This takes a while. Feel free to come back here after finishing
do_HP_optimization = False

In [ ]:
# RandomSearchCV for advanced HPO
import scipy.stats as stats

if do_HP_optimization:
    from sklearn.model_selection import RandomizedSearchCV

    # specify parameters, range, and distributions to sample from
    param_dist_XGB = {
        'max_depth': stats.randint(3, 12),  # default 6
        'n_estimators': stats.randint(300, 800),  # default 100
        'learning_rate': stats.uniform(0.1, 0.5),
    }  # def 0.3

    # default CV is 5 fold, reduce to 2 for speed concern
    gsearch = RandomizedSearchCV(
        estimator=XGBClassifier(tree_method="hist", use_label_encoder=False, eval_metric='logloss'),
        param_distributions=param_dist_XGB,
        scoring='roc_auc',
        n_iter=10,
        cv=2,
    )
    gsearch.fit(X_train, y_train, sample_weight=weights_train)

    print("Best parameters : ", gsearch.best_params_)
    print("Best score (on train dataset CV) : ", gsearch.best_score_)

    y_pred_gs = gsearch.predict_proba(X_test)[:, 1]
    print(
        "... corresponding score on test dataset : ",
        roc_auc_score(y_true=y_test, y_score=y_pred_gs, sample_weight=weights_test),
    )
    dfsearch = pd.DataFrame.from_dict(gsearch.cv_results_)
    display(dfsearch)

In [ ]:
if do_HP_optimization:
    dfsearch.plot.scatter("param_n_estimators", "mean_test_score")
    dfsearch.plot.scatter("param_max_depth", "mean_test_score")
    dfsearch.plot.scatter("param_learning_rate", "mean_test_score")

# Learning Curve
This could be done with sklearn  [learning_curve](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.learning_curve.html).
However, older versions cannot handle weights, and therefore do not allow you to control testing dataset size. Need to check newer versions



In [ ]:
from sklearn.metrics import roc_curve

Do_Learning_Curve = True

In [ ]:
if Do_Learning_Curve:
    #train_sizes = [0.01, 0.05, 0.1, 0.2, 0.5, 0.75, 1]
    #train_sizes = [0.001, 0.003, 0.005, 0.01, 0.05, 0.1, 0.2]
    train_sizes = [0.005, 0.02, 0.05, 0.1, 0.2, 0.5, 1.]
    ntrains = []
    test_aucs = []
    train_aucs = []
    times = []
    fpr = []
    tpr = []

    for train_size in train_sizes:
        ntrain = int(len(X_train) * train_size)
        print("training with ", ntrain, " events")
        ntrains += [ntrain]
        starting_time = time.time()

        # train using the first ntrain event of the training dataset
        gbm.fit(X_train[:ntrain,], y_train[:ntrain], sample_weight=weights_train[:ntrain])
        training_time = time.time() - starting_time
        times += [training_time]

        # score on test dataset (always the same)
        y_pred_gbm = gbm.predict_proba(X_test)[:, 1]
        auc_test_gbm = roc_auc_score(y_true=y_test[posWeightsTest], 
                                     y_score=y_pred_gbm[posWeightsTest], 
                                     sample_weight=weights_test[posWeightsTest])
        test_aucs += [auc_test_gbm]

        # score on the train dataset
        y_train_gbm = gbm.predict_proba(X_train[:ntrain])[:, 1]
        auc_train_gbm = roc_auc_score(y_true=y_train[:ntrain][weights_train[:ntrain]>0], 
                                      y_score=y_train_gbm[weights_train[:ntrain]>0], 
                                      sample_weight=weights_train[:ntrain][weights_train[:ntrain]>0])
        train_aucs += [auc_train_gbm]
        dflearning = pd.DataFrame({"Ntraining": ntrains, "test_auc": test_aucs, "train_auc": train_aucs, "time": times})

        #temp_fpr, temp_tpr, _ = roc_curve(y_true=y_train[:ntrain], y_score=y_train_gbm, sample_weight=weights_train[:ntrain])
        temp_fpr, temp_tpr, _ = roc_curve(y_true=y_test, y_score=y_pred_gbm, sample_weight=weights_test)
        fpr += [temp_fpr]
        tpr += [temp_tpr]

    display(dflearning)

In [ ]:
if Do_Learning_Curve:
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    lw = 2
    # axes[0].plot(fpr_xgb, tpr_xgb, color='blue',lw=lw, label='XGBoost (AUC  = {})'.format(np.round(auc_test_xgb,decimals=2)))
    axes[0].plot(fpr[0], tpr[0], color='blue', lw=lw, label='NTrain = %i' % (ntrains[0]))
    axes[0].plot(fpr[1], tpr[1], color='red', lw=lw, label='NTrain = %i' % (ntrains[1]))
    axes[0].plot(fpr[3], tpr[3], color='green', lw=lw, label='NTrain = %i' % (ntrains[3]))
    axes[0].plot(fpr[4], tpr[4], color='orange', lw=lw, label='NTrain = %i' % (ntrains[4]))
    axes[0].plot(fpr[6], tpr[6], color='black', lw=lw, label='NTrain = %i' % (ntrains[6]))

    axes[0].legend()
    axes[0].set_xlabel('False Positive Rate')
    axes[0].set_ylabel('True Positive Rate')
    axes[0].set_title('Receiver Operating Characteristic')

    axes[1].set_title("Test AUC vs. NTraining")
    dflearning.plot.scatter("Ntraining", "test_auc", ax=axes[1])
    # focus on the last point
    # dflearning[4:].plot.scatter("Ntraining","test_auc")

# Feature importance
Feature importance allows to display the importance of each feature without rerunnning the training. 

It is obtained from internal algorithm quantities, like number of time a feature is used to define leaf, and the information gained from the nodes that use that feature.

Magnitude is arbitrary. It not always a reliable indication of which feature is the most discriminant.

In [ ]:
gbm._Booster.dump_model()['feature_names']

In [ ]:
#plt.bar(data.columns.values, xgb.feature_importances_)
lightgbm.plot_importance(booster=gbm, 
                         importance_type='gain',
                        title= "Feature importance (gain)",
                        )
#plt.xticks(rotation=90)
#plt.title("Feature importances Light GBM")

# Permutation importance

A better way to show the importance of each feature is Permutation Importance, where each feature in turn is replaced by an instance of that feature from another event (effectively switching it off by randomising).
In particular it allows one to : 
   * display directly the loss in whatever criteria (ROC auc, asimov significance) when the feature is switched off
   * display the feature importance for a specific subset (for example the most signal like)
   * it can even display which feature has the larges impact on systematics


However, report can be misleading in case of highly correlated variables. 
   


In [ ]:
from permutationimportancephysics.PermutationImportance import PermulationImportance
pi = PermulationImportance(model=gbm, X=X_test,y=y_test,weights=weights_test,n_iterations=3,usePredict_poba=True,
                      scoreFunction="amsasimov", colNames=data.columns)
plt = pi.plotBars()
plt.show()

### Score Plot without Renormalizing

In [ ]:
compare_train_test(
    y_pred_train_gbm,
    y_train,
    y_pred_gbm,
    y_test,
    xlabel="LGBM score",
    ylabel="Expected number of events",
    title="LGBM",
    weights_train=weights_train.values,
    weights_test=weights_test.values,
    density=False,
)

# Extra BDT Examples

___
__Example with sklearn (but without weights and with no fixed test dataset):__

In [ ]:
# change this cell from 'raw' to 'code'
from sklearn.model_selection import learning_curve

train_sizes, train_scores, test_scores = learning_curve(
    XGBClassifier(tree_method="hist", use_label_encoder=False, eval_metric='logloss', n_estimators=10),
    X_train,
    y_train,
    train_sizes=[0.01, 0.05, 0.1, 0.2, 0.5, 0.75, 1],
    scoring='roc_auc',
    cv=5,
)

___

# Train BDT using [XGBoost](https://arxiv.org/abs/1603.02754)

In [ ]:
np.random.seed(31415)  # set the random seed

from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score  # for binary classification if x > 0.5 -> 1 else -> 0

xgb = XGBClassifier(tree_method="hist", use_label_encoder=False, eval_metric='logloss')
# tree_method="hist" is 10 times faster, however less robust against awkwards features (not a bad idea to double check without it)
# can even try tree_method="gpu_hist" if proper GPU installation
# use_label_encoder and eval_metric to silence warning in 1.3.0

# HPO (==Hyper Parameter Optimization), check on the web https://xgboost.readthedocs.io/ for other parameters
# xgb = XGBClassifier(tree_method="hist",use_label_encoder=False,max_depth=10,n_estimators=100)


starting_time = time.time()

xgb.fit(X_train, y_train.values, sample_weight=weights_train.values)  # note that XGB 1.3.X requires positive weight

training_time = time.time() - starting_time
print("Training time:", training_time)

y_pred_xgb = xgb.predict_proba(X_test)[:, 1]
y_pred_xgb = y_pred_xgb.ravel()
y_pred_train_xgb = xgb.predict_proba(X_train)[:, 1].ravel()
auc_test_xgb = roc_auc_score(y_true=y_test, y_score=y_pred_xgb, sample_weight=weights_test)
print("auc test:", auc_test_xgb)
print("auc train:", roc_auc_score(y_true=y_train.values, y_score=y_pred_train_xgb, sample_weight=weights_train), "\n")

print("auc test without weights", roc_auc_score(y_true=y_test, y_score=y_pred_xgb))

In [ ]:
# sum S & B after certain thresholds
int_pred_test_sig_xgb = [weights_test[(y_test == 1) & (y_pred_xgb > th_cut)].sum() for th_cut in np.linspace(0, 1, num=50)]
int_pred_test_bkg_xgb = [weights_test[(y_test == 0) & (y_pred_xgb > th_cut)].sum() for th_cut in np.linspace(0, 1, num=50)]

vamsasimov_xgb = [amsasimov(sumsig, sumbkg) for (sumsig, sumbkg) in zip(int_pred_test_sig_xgb, int_pred_test_bkg_xgb)]
significance_xgb = max(vamsasimov_xgb)  # finds the maximum significance. Which threshold results in this significance?
Z = significance_xgb
print("Z:", Z)

## SKLearn GBDT

In [ ]:
from sklearn import ensemble

# possible parameters, just take the default
# original_params = {
#    "n_estimators": 400,
#    "max_leaf_nodes": 4,
#    "max_depth": None,
#    "random_state": 2,
#    "min_samples_split": 5,
# }

skgb = ensemble.HistGradientBoostingClassifier()


starting_time = time.time()

skgb.fit(X_train, y_train.values, sample_weight=weights_train.values)


training_time = time.time() - starting_time
print("Training time:", training_time)

y_pred_skgb = skgb.predict_proba(X_test)[:, 1]
y_pred_skgb = y_pred_skgb.ravel()
y_pred_train_skgb = skgb.predict_proba(X_train)[:, 1].ravel()
auc_test_skgb = roc_auc_score(y_true=y_test, y_score=y_pred_skgb, sample_weight=weights_test)
print("auc test:", auc_test_skgb)
print("auc train:", roc_auc_score(y_true=y_train.values, y_score=y_pred_train_skgb, sample_weight=weights_train))

int_pred_test_sig_skgb = [weights_test[(y_test == 1) & (y_pred_skgb > th_cut)].sum() for th_cut in np.linspace(0, 1, num=50)]
int_pred_test_bkg_skgb = [weights_test[(y_test == 0) & (y_pred_skgb > th_cut)].sum() for th_cut in np.linspace(0, 1, num=50)]

vamsasimov_skgb = [amsasimov(sumsig, sumbkg) for (sumsig, sumbkg) in zip(int_pred_test_sig_skgb, int_pred_test_bkg_skgb)]
significance_skgb = max(vamsasimov_skgb)
Z = significance_skgb
print("Z:", Z)

In [ ]:
print('Best significance found are:')
print('XGBoost : ', significance_xgb)
print('LightGBM: ', significance_gbm)
print('sklearn: ', significance_skgb)


print('Best auc test found are:')
print('XGBoost: ', roc_auc_score(y_true=y_test.values, y_score=y_pred_xgb, sample_weight=weights_test))
print('LightGBM: ', roc_auc_score(y_true=y_test.values, y_score=y_pred_gbm, sample_weight=weights_test))
print('sklearn: ', roc_auc_score(y_true=y_test.values, y_score=y_pred_skgb, sample_weight=weights_test))

## Some nice plots 

### load score plotting function

In [ ]:
# Plot score for signal and background, comparing training and testing
def compare_train_test(
    y_pred_train,
    y_train,
    y_pred,
    y_test,
    high_low=(0, 1),
    bins=30,
    xlabel="",
    ylabel="Arbitrary units",
    title="",
    weights_train=np.array([]),
    weights_test=np.array([]),
    density=True,
):
    if weights_train.size != 0:
        weights_train_signal = weights_train[y_train == 1]
        weights_train_background = weights_train[y_train == 0]
    else:
        weights_train_signal = None
        weights_train_background = None
    plt.hist(
        y_pred_train[y_train == 1],
        color='r',
        alpha=0.5,
        range=high_low,
        bins=bins,
        histtype='stepfilled',
        density=density,
        label='S (train)',
        weights=weights_train_signal,
    )  # alpha is transparancy
    plt.hist(
        y_pred_train[y_train == 0],
        color='b',
        alpha=0.5,
        range=high_low,
        bins=bins,
        histtype='stepfilled',
        density=density,
        label='B (train)',
        weights=weights_train_background,
    )

    if weights_test.size != 0:
        weights_test_signal = weights_test[y_test == 1]
        weights_test_background = weights_test[y_test == 0]
    else:
        weights_test_signal = None
        weights_test_background = None
    hist, bins = np.histogram(y_pred[y_test == 1], bins=bins, range=high_low, density=density, weights=weights_test_signal)
    scale = len(y_pred[y_test == 1]) / sum(hist)
    err = np.sqrt(hist * scale) / scale

    center = (bins[:-1] + bins[1:]) / 2
    plt.errorbar(center, hist, yerr=err, fmt='o', c='r', label='S (test)')

    hist, bins = np.histogram(
        y_pred[y_test == 0], bins=bins, range=high_low, density=density, weights=weights_test_background
    )
    scale = len(y_pred[y_test == 0]) / sum(hist)
    err = np.sqrt(hist * scale) / scale

    center = (bins[:-1] + bins[1:]) / 2
    plt.errorbar(center, hist, yerr=err, fmt='o', c='b', label='B (test)')
    plt.title(title)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(loc='best')

In [ ]:
compare_train_test(
    y_pred_train_xgb,
    y_train,
    y_pred_xgb,
    y_test,
    xlabel="XGboost score",
    title="XGboost",
    weights_train=weights_train.values,
    weights_test=weights_test.values,
)
plt.savefig("Score_BDT_XGBoost_Hist.pdf")
plt.show()
compare_train_test(
    y_pred_train_gbm,
    y_train,
    y_pred_gbm,
    y_test,
    xlabel="LightGBM score",
    title="LightGBM",
    weights_train=weights_train.values,
    weights_test=weights_test.values,
)
plt.savefig("Score_BDT_LightGBM.pdf")
plt.show()
compare_train_test(
    y_pred_train_skgb,
    y_train,
    y_pred_skgb,
    y_test,
    xlabel="sklearn score",
    title="sklearn",
    weights_train=weights_train.values,
    weights_test=weights_test.values,
)
plt.savefig("Score_BDT_sklearn.pdf")
plt.show()

### Plot without renormalising

In [ ]:
compare_train_test(
    y_pred_train_xgb,
    y_train,
    y_pred_xgb,
    y_test,
    xlabel="XGboost score",
    ylabel="Expected number of events",
    title="XGboost",
    weights_train=weights_train.values,
    weights_test=weights_test.values,
    density=False,
)
plt.savefig("Score_BDT_XGBoost_Hist.pdf")
plt.show()

### ROC curve

In [ ]:
from sklearn.metrics import roc_curve

lw = 2

fpr_xgb, tpr_xgb, _ = roc_curve(y_true=y_test, y_score=y_pred_xgb, sample_weight=weights_test.values)
fpr_gbm, tpr_gbm, _ = roc_curve(y_true=y_test, y_score=y_pred_gbm, sample_weight=weights_test.values)
fpr_skgb, tpr_skgb, _ = roc_curve(y_true=y_test, y_score=y_pred_skgb, sample_weight=weights_test.values)


plt.plot(fpr_xgb, tpr_xgb, color='darkgreen', lw=lw, label='XGBoost (AUC  = {})'.format(np.round(auc_test_xgb, decimals=2)))
plt.plot(
    fpr_skgb, tpr_skgb, color='darkblue', lw=lw, label='sklearn (AUC  = {})'.format(np.round(auc_test_skgb, decimals=2))
)
plt.plot(
    fpr_gbm, tpr_gbm, color='darkorange', lw=lw, label='LightGBM (AUC  = {})'.format(np.round(auc_test_gbm, decimals=2))
)


plt.plot([0, 1], [0, 1], color='navy', lw=lw, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
# import os
# new_dir = "Plots/Comparing"
# if not os.path.isdir(new_dir):
#    os.mkdir(new_dir)
plt.savefig("ROC_comparing.pdf")
plt.show()  # blue line = random classification -> maximize true positive rate while miniize false positive rate

### Significance curve

In [ ]:
plt.plot(np.linspace(0, 1, num=50), vamsasimov_xgb, label='XGBoost (Z = {})'.format(np.round(significance_xgb, decimals=2)))
plt.plot(
    np.linspace(0, 1, num=50), vamsasimov_skgb, label='sklearn (Z = {})'.format(np.round(significance_skgb, decimals=2))
)
plt.plot(np.linspace(0, 1, num=50), vamsasimov_gbm, label='LightGBM (Z = {})'.format(np.round(significance_gbm, decimals=2)))


plt.title("BDT Significance")
plt.xlabel("Threshold")
plt.ylabel("Significance")
plt.legend()
plt.savefig("Significance_comparing.pdf")
plt.show()

## XGBoost and LightGBM Feature Importance
Feature importance allows to display the importance of each feature without rerunnning the training. It is obtained from internal algorithm quantities, like number of time a feature is used to define leaf, and the information gained from the nodes that use that feature. Magnitude is arbitrary. It not always a reliable indication of which feature is the most discriminant.

In [ ]:
plt.bar(data.columns.values, xgb.feature_importances_)
plt.xticks(rotation=90)
plt.title("Feature importances XGBoost Hist")
# plt.savefig(new_dir + "/VarImp_BDT_XGBoost_Hist.pdf",bbox_inches='tight')
plt.show()
plt.bar(data.columns.values, gbm.feature_importances_) # not gain
plt.xticks(rotation=90)
plt.title("Feature importances LightGBM")
# plt.savefig(new_dir + "/VarImp_BDT_LightGBM.pdf",bbox_inches='tight')
plt.show()

##  Model serialisation
It is useful to be able to save a model in order to apply it without retraining. There are many ways to do it. One is to save the whole python object with joblib (beware this is not safe if the software evolves). Another is to used dedicated serialisation like the one proposed by xgboost.


In [ ]:
#WARNING : StandardScaler has not been saved
# one can look into sklearn pipeline
if False:
    import joblib
    myxgb = XGBClassifier(tree_method="hist",use_label_encoder=False,eval_metric='logloss',n_estimators=5)
    myxgb.fit(X_train, y_train, sample_weight=weights_train)

    auc_test_xgb = roc_auc_score(y_true=y_test, y_score=myxgb.predict_proba(X_test)[:,1],sample_weight=weights_test)
 
    # save model
    myxgb.save_model("XGBoost.json")


    # save python object
    joblib.dump(myxgb, "myxgb.dat")

    print ("myxgb score",auc_test_xgb)

    del myxgb # delete xgb object

    # reload model
    myxgb_reloaded_from_model =XGBClassifier()
    myxgb_reloaded_from_model.load_model("XGBoost.json")
    print ("myxgb reloaded from model",
           roc_auc_score(y_true=y_test, 
                         y_score=myxgb_reloaded_from_model.predict_proba(X_test)[:,1],sample_weight=weights_test)
          )


    # reload object
    myxgb_reloaded_from_joblib=joblib.load("myxgb.dat")
    print ("myxgb reloaded from object",
           roc_auc_score(y_true=y_test, 
                         y_score=myxgb_reloaded_from_joblib.predict_proba(X_test)[:,1],sample_weight=weights_test)
          )
    # dump json file
    !python -m json.tool XGBoost.json





